# Sanan ennustaja – Koneoppimismalleihin perustuva sananjatkaja

Tässä projektissa kehitetään järjestelmä, joka ennustaa käyttäjän kirjoittaman tekstin seuraavaa sanaa. Käytössä on neljä eri GPT-2-kielimalliin perustuvaa vaihtoehtoa:
- kaksi englanninkielistä esikoulutettua mallia (DistilGPT2 ja GPT2-Medium),
- yksi suomenkielinen GPT-2-malli (Finnish-NLP), sekä
- yksi itse koulutettu malli, joka on hienosäädetty käyttäjän omaan PDF-muotoiseen tekstiin perustuen.

Projektin vaiheet:
- PDF-muotoisen tekstin purku ja esikäsittely
- GPT-2-mallin hienosäätö omalla tekstillä
- Mallien lataus ja käyttö interaktiivisessa käyttöliittymässä (Tkinter)
- Mallien vertailu: esikoulutetut mallit vs. oma koneoppimisella koulutettu malli

Projektin päätavoitteena on tutkia, kuinka hyvin koneoppimismalli voi oppia yksilöllistä kieltä ja tyyliä, ja hyödyntää sitä sanan ennustamiseen kirjoittamisen yhteydessä. Lisäksi vertaillaan eri mallien kykyä tuottaa loogisia ja kontekstiin sopivia sanajatkoja.

Notebook sisältää:
- Koodin PDF-datan käsittelyyn
- Koulutuksen koneoppimismallilla (GPT-2 hienosäätö)
- Mallien käytön ja testauksen
- Selitykset vaiheista ja lopputulosten arvioinnin

Lisäksi mukana on erillinen Python-pohjainen graafinen käyttöliittymä (sananarvaus.py), jossa käyttäjä voi kirjoittaa tekstiä ja saada mallin antamia ehdotuksia sanan jatkoksi. Mallin voi valita käyttöliittymästä painikkeilla.

**Työkalut:** Python, PyPDF2, Hugging Face Transformers, datasets, tkinter, reportlab

**Kielimallit:** GPT-2 (generatiiviset kielimallit), mukaan lukien oma hienosäädetty versio

**Tiedostot:**
- `1_extract_pdf.py` – PDF-tekstin purku
- `2_finetune_gpt2.py` – Mallin koulutus
- `sananarvaus.py` – Käyttöliittymä


## 2. PDF-tekstin purku koulutusaineistoksi
Tässä vaiheessa luemme PDF-tiedoston, joka sisältää kirjoitettua tekstiä. Tavoitteena on muuntaa sisältö koulutukseen sopivaksi raakatiedoksi.

In [ ]:
import PyPDF2
import os

# Funktio tekstin purkamiseen PDF-tiedostosta
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# PDF-tiedoston polku (varmista, että tiedosto on olemassa)
pdf_path = os.path.join("oma_gpt2_malli", "teksti_1.pdf")
raw_text = extract_text_from_pdf(pdf_path)

# Tallennetaan teksti koulutustiedostoksi
with open("train.txt", "w", encoding="utf-8") as f:
    f.write(raw_text)

print("PDF-sisältö tallennettu train.txt-tiedostoon")

# 3. GPT-2-mallin hienosäätö omalla tekstillä

In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Ladataan koulutusdata
dataset = load_dataset("text", data_files={"train": "train.txt"})

# Tokenisaattorin lataus ja asetukset
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# GPT-2-mallin lataus
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Datan valmistelu mallille
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Koulutusparametrit
training_args = TrainingArguments(
    output_dir="oma_gpt2_malli",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    prediction_loss_only=True,
)

# Trainerin alustaminen
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Mallin koulutus ja tallennus
trainer.train()
trainer.save_model("oma_gpt2_malli")
tokenizer.save_pretrained("oma_gpt2_malli")

print("Oma kielimalli tallennettu kansioon: oma_gpt2_malli")

#  4. Käyttöliittymän koodin rakenne (Tkinter)


Tässä osiossa kuvataan graafisen käyttöliittymän pääkomponentit, jotka on toteutettu tiedostossa sananarvaus.py.


# 4.1 Kirjastojen tuonti ja asetukset

In [ ]:
import tkinter as tk
from transformers import pipeline, set_seed
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from tkinter import filedialog

current_prediction = ""

# Globaali generointimalli
generator = None

# Asetetaan satunnaissiementä
def initialize_seed():
    set_seed(42)

# 4.2 Mallien valinta ja lataus

In [ ]:
MODELS = {
    "DistilGPT2 (EN, kevyt)": "distilgpt2",
    "GPT2-Medium (EN, parempi)": "gpt2-medium",
    "Suomi GPT2 (Finnish-NLP)": "Finnish-NLP/gpt2-finnish",
    "Oma malli (hienosäädetty)": "oma_gpt2_malli"
}

def load_model(model_name):
    global generator
    status_label.config(text=f"Ladataan mallia: {model_name}...")
    root.update()
    try:
        generator = pipeline("text-generation", model=model_name)
        status_label.config(text=f"Aktiivinen malli: {model_name}")
    except Exception as e:
        status_label.config(text=f"Virhe ladattaessa: {e}")

# 4.3 PDF-tallennusfunktio

In [ ]:
def save_text_as_pdf():
    text = text_input.get("1.0", tk.END).strip()
    if not text:
        status_label.config(text="Ei tallennettavaa tekstiä.")
        return

    file_path = filedialog.asksaveasfilename(
        defaultextension=".pdf",
        filetypes=[("PDF Files", "*.pdf")],
        title="Tallenna PDF-tiedostona"
    )
    if not file_path:
        return

    try:
        c = canvas.Canvas(file_path, pagesize=A4)
        width, height = A4
        margin = 50
        y = height - margin
        lines = text.split('\n')

        for line in lines:
            c.drawString(margin, y, line)
            y -= 15
            if y < margin:
                c.showPage()
                y = height - margin

        c.save()
        status_label.config(text=f"PDF tallennettu: {file_path}")
    except Exception as e:
        status_label.config(text=f"Virhe tallennettaessa: {e}")


# 4.4 Ennustaminen ja näppäintapahtumat

In [ ]:
def predict_next_word(prompt_text):
    global current_prediction
    if not generator:
        current_prediction = ""
        return "Ei mallia ladattu"
    try:
        generated = generator(
            prompt_text,
            max_new_tokens=5,
            num_return_sequences=1,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True
        )
        generated_text = generated[0]['generated_text']
        next_words = generated_text[len(prompt_text):].strip().split()
        for word in next_words:
            if word.isalpha():
                current_prediction = word
                return word
        current_prediction = next_words[0] if next_words else ""
        return current_prediction
    except Exception as e:
        current_prediction = ""
        return f"Virhe: {e}"

def on_key_release(event):
    input_text = text_input.get("1.0", tk.END).strip()
    if input_text:
        next_word = predict_next_word(input_text)
        prediction_label.config(text=f"Ehdotus: {next_word}")
    else:
        prediction_label.config(text="Ehdotus: ...")

def on_tab_press(event):
    global current_prediction
    if current_prediction:
        text_input.insert(tk.INSERT, " " + current_prediction)
        current_prediction = ""
        prediction_label.config(text="Ehdotus: ...")
        return "break"  # estää oletustoiminnon

# 4.5 Käyttöliittymän rakenne

In [ ]:
# Luo ikkuna
root = tk.Tk()
root.title("Seuraavan sanan ennustaja")
root.configure(bg="#e6f0fa")

label_font = ("Helvetica", 14)
button_font = ("Helvetica", 11)

text_input = tk.Text(root, height=10, width=60, font=("Helvetica", 12), bg="#ffffff", bd=2, relief="groove")
text_input.pack(padx=20, pady=(20,10))
text_input.bind("<KeyRelease>", on_key_release)
text_input.bind("<Tab>", on_tab_press)

prediction_label = tk.Label(root, text="Ehdotus: ...", font=label_font, bg="#e6f0fa", fg="#003366")
prediction_label.pack(pady=(5, 15))

save_button = tk.Button(root, text="Tallenna PDF", command=save_text_as_pdf, font=button_font, bg="#d9edf7", relief="raised")
save_button.pack(pady=(0, 10))

button_frame = tk.Frame(root, bg="#e6f0fa")
button_frame.pack(pady=(0, 15))

for label, model_name in MODELS.items():
    btn = tk.Button(button_frame, text=label, command=lambda m=model_name: load_model(m),
                    font=button_font, bg="#cce5ff", relief="groove")
    btn.pack(side=tk.LEFT, padx=5)

status_label = tk.Label(root, text="Valitse kielimalli.", fg="#004085", bg="#e6f0fa", font=("Helvetica", 10, "italic"))
status_label.pack(pady=(0, 10))

# Käynnistä ikkuna
root.mainloop()


# 5. Johtopäätökset

Projektin aikana tutkittiin neljän eri GPT-2-pohjaisen mallin kykyä ennustaa seuraava sana kirjoitetun tekstin perusteella. Hienosäädetty oma malli antoi selvästi parempia tuloksia erityisesti silloin, kun syötteenä oli sen koulutuksessa käytettyä tyyliä.

Jatkokehityksessä voisi:
- kasvattaa koulutusdataa useammalla PDF-tiedostolla
- säätää hyperparametreja tarkemmin
- lisätä automaattisen arviointimittarin (BLEU, perplexity)